In [ ]:
import sqlite3
import pandas as pd

In [ ]:
df = pd.read_csv("shipments.csv")

In [ ]:
df

,order_id,delivery_date,route_id,city,delivery_cost,rider_payout,revenue,distance_km
0,1,2024-04-12,159.0,Delhi,118.62,81.16,202.29,17.70
1,2,2024-12-14,158.0,Kolkata,53.61,32.46,92.57,20.91
2,3,2024-09-27,169.0,Hyderabad,60.83,39.86,69.50,21.63
3,4,2024-04-16,133.0,Kolkata,72.83,56.08,104.48,23.67
4,5,2024-03-12,133.0,Hyderabad,71.22,50.47,119.12,12.02
...,...,...,...,...,...,...,...,...
194813,194814,2024-08-10,193.0,Chennai,56.24,35.99,85.35,20.10
194814,194815,2024-08-28,130.0,Mumbai,63.19,40.42,70.13,23.95
194815,194816,2024-07-06,180.0,Delhi,74.78,52.07,110.27,17.50
194816,194817,2024-09-15,178.0,Bengaluru,71.49,35.76,109.30,15.73


In [ ]:
conn = sqlite3.connect("logistics_finance.db")

In [ ]:
df.to_sql("shipments", conn, if_exists="replace", index=False)

194818

In [ ]:

#  Run SQL queries directly
query = """
SELECT city,
       AVG(delivery_cost) AS avg_cost,
       AVG((revenue - delivery_cost)/revenue*100) AS avg_margin
FROM shipments
GROUP BY city
ORDER BY avg_margin DESC
"""
result = pd.read_sql_query(query, conn)

In [ ]:
result

,city,avg_cost,avg_margin
0,Bengaluru,80.207565,29.660623
1,Kolkata,79.903144,29.645561
2,Chennai,80.394388,29.639351
3,Mumbai,79.871213,29.632439
4,Delhi,79.832495,29.603478
5,Hyderabad,79.876324,29.555042
6,Hyderab,NaN,NaN


In [ ]:
query = """
SELECT AVG(delivery_cost) AS avg_cost_per_delivery
FROM shipments;
"""
result = pd.read_sql_query(query, conn)
display(result)

,avg_cost_per_delivery
0,79.999009


In [ ]:
query = """
SELECT AVG((revenue - delivery_cost) / revenue * 100) AS avg_gross_margin_pct
FROM shipments;
"""
result = pd.read_sql_query(query, conn)
display(result)

,avg_gross_margin_pct
0,29.625949


In [ ]:
query = """
SELECT SUM(revenue - (delivery_cost + rider_payout)) AS total_net_profit
FROM shipments;
"""
result = pd.read_sql_query(query, conn)
display(result)

,total_net_profit
0,-3128875.1


In [ ]:
query = """
SELECT
    route_id,
    COUNT(*) AS total_orders,
    SUM(revenue) AS total_revenue,
    SUM(delivery_cost) AS total_cost,
    SUM(revenue - delivery_cost) AS gross_profit
FROM shipments
GROUP BY route_id
ORDER BY gross_profit DESC;
"""
result = pd.read_sql_query(query, conn)
display(result)

,route_id,total_orders,total_revenue,total_cost,gross_profit
0,180.0,2023,238127.79,163557.59,74570.20
1,115.0,2021,237490.77,163393.39,74097.38
2,133.0,2037,238216.11,164355.66,73860.45
3,156.0,2023,237887.71,164103.85,73783.86
4,197.0,2009,233844.51,160324.52,73519.99
...,...,...,...,...,...
96,118.0,1852,214038.14,147135.04,66903.10
97,128.0,1841,214857.00,148040.41,66816.59
98,188.0,1869,215742.33,149256.70,66485.63
99,134.0,1879,214740.80,148835.53,65905.27


In [ ]:
query = """
SELECT
    city,
    SUM(revenue) AS total_revenue,
    SUM(delivery_cost) AS total_cost,
    SUM(revenue - delivery_cost) AS gross_profit
FROM shipments
GROUP BY city
ORDER BY gross_profit DESC;
"""
result = pd.read_sql_query(query, conn)
display(result)

,city,total_revenue,total_cost,gross_profit
0,Chennai,3805454.62,2624472.02,1180982.60
1,Hyderabad,3790534.13,2614534.43,1175999.70
2,Bengaluru,3771438.72,2601254.02,1170184.70
3,Kolkata,3752109.50,2589486.15,1162623.35
4,Mumbai,3736687.75,2577485.82,1159201.93
5,Delhi,3736560.47,2577934.57,1158625.90
6,None,NaN,NaN,NaN


In [ ]:
query = """
SELECT
    route_id,
    AVG(delivery_cost) AS avg_cost
FROM shipments
GROUP BY route_id
ORDER BY avg_cost DESC
LIMIT 10;
"""
result = pd.read_sql_query(query, conn)
display(result)

,route_id,avg_cost
0,109.0,81.558893
1,168.0,81.133981
2,156.0,81.119056
3,126.0,80.919875
4,199.0,80.882713
5,180.0,80.849031
6,115.0,80.847793
7,185.0,80.760721
8,108.0,80.725479
9,148.0,80.700795


In [ ]:
query = """
SELECT
    delivery_date,
    city,
    route_id,
    SUM(revenue) AS total_revenue,
    SUM(delivery_cost) AS total_cost,
    SUM(revenue - delivery_cost) AS gross_profit,
    SUM(revenue - (delivery_cost + rider_payout)) AS net_profit
FROM shipments
GROUP BY delivery_date, city, route_id
ORDER BY delivery_date;
"""
result = pd.read_sql_query(query, conn)
display(result)

,delivery_date,city,route_id,total_revenue,total_cost,gross_profit,net_profit
0,2024,None,NaN,NaN,NaN,NaN,NaN
1,2024-01-01,Bengaluru,100.0,187.84,119.92,67.92,-7.19
2,2024-01-01,Bengaluru,101.0,126.82,78.49,48.33,-12.48
3,2024-01-01,Bengaluru,103.0,174.31,114.51,59.80,-15.59
4,2024-01-01,Bengaluru,104.0,187.39,115.72,71.67,-15.42
...,...,...,...,...,...,...,...
128560,2024-12-30,Mumbai,183.0,169.03,96.66,72.37,4.40
128561,2024-12-30,Mumbai,184.0,257.58,178.35,79.23,-33.48
128562,2024-12-30,Mumbai,185.0,241.60,157.25,84.35,-33.93
128563,2024-12-30,Mumbai,191.0,79.20,46.58,32.62,1.55
